In [609]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib import style
from scipy.stats import norm
import plotly.graph_objects as go
import numpy as np
import inquirer
import pickle
# allows currency formatting
import locale
locale.setlocale(locale.LC_ALL, 'en_CA.UTF-8')
%matplotlib inline
style.use('ggplot')

In [ ]:
#Ask for user name 
#Ask for user age
#Ask for how long of investment
# Ask for conservative: 30%, moderate: 50%, risky 80%
# Give total amount stocks investing in
# Build your portfolio
# standard 10 stocks will increase as upgraded
# conservative 3 growth, 7 value - out of Top 20 for both growth and value
# moderate 5 growth, 5 value - out of Top 20 for both growth and value
# risky 8 growth, 2 value - out of Top 20 for both growth and value
# save the similar stocks in a dataframe
# give their amount they would spend each year 
# give possible outcomes

In [610]:
cluster_rec = pd.read_csv('rec_csvs/clusters_rec.csv')
growth_stocks = pd.read_csv('rec_csvs/growth_stocks.csv')
value_stocks = pd.read_csv('rec_csvs/value_stocks.csv')
predictions = pd.read_csv('rec_csvs/prediction_gv.csv')
growth_stocks = growth_stocks.rename({'0':'symbol'}, axis = 1)
value_stocks = value_stocks.rename({'0':'symbol'}, axis = 1)

In [611]:
cluster_rec = cluster_rec.drop(columns= 'Unnamed: 0')

In [612]:
# cluster_ticker = pd.DataFrame(cluster_rec['symbol'])
# cluster_ticker.to_csv('/Users/flatironschool/UrPortfolio/Classification_Models/clusters_ticker.csv', index = False)

###  User Input - What is stock is Growth or Value 

In [613]:
stockz = input('Input a Stock to know if it is Growth or Value: ').upper()
prede = predictions[predictions['symbol'] == stockz]
growth = test['Growth'].sum()
value = test['Value'].sum()
if growth > value:
    print(f'Through our fundamental anaylsis: {stockz} is a Growth Stock')
else:
    print(f'Through our fundamental anaylsis: {stockz} is a Value Stock')

Input a Stock to know if it is Growth or Value: a
Through our fundamental anaylsis: A is a Value Stock


In [614]:
matching = cluster_rec.set_index('symbol')

### Top 20 Value Choices for User

In [615]:
value_choices = value_stocks.set_index('symbol')
value_choices = value_choices.merge(matching, right_index = True, left_index = True)
# value_choices = value_choices.drop(columns ='Unnamed: 0')
conservative_value_stocks = value_choices[value_choices['yr_variance'] <= .45].reset_index()
moderate_value_stocks = value_choices[(value_choices['yr_variance'] <= .75) & (value_choices['yr_variance']>= .45 )].reset_index()
risky_value_stocks = value_choices[(value_choices['yr_variance'] <= 1.) & (value_choices['yr_variance'] >= .75)].reset_index()

In [616]:
top_20_conservative_value = conservative_value_stocks.sort_values('avg_yr_returns', ascending = False).head(20)
top_20_moderate_value = moderate_value_stocks.sort_values('avg_yr_returns', ascending = False).head(20)
top_20_risky_value = risky_value_stocks.sort_values('avg_yr_returns', ascending = False).head(20)

### Top 20 Growth Choices For User

In [617]:
growth_choices = growth_stocks.set_index('symbol')
growth_choices = growth_choices.merge(matching, right_index = True, left_index = True)
# growth_choices = growth_choices.drop(columns ='Unnamed: 0')
conservative_growth_stocks = growth_choices[growth_choices['yr_variance'] <= .45].reset_index()
moderate_growth_stocks = growth_choices[(growth_choices['yr_variance'] <= .75) & (growth_choices['yr_variance']>= .45 )].reset_index()
risky_growth_stocks = growth_choices[(growth_choices['yr_variance'] <= 1.) & (growth_choices['yr_variance'] >= .75)].reset_index()


In [618]:
top_20_conservative_growth = conservative_growth_stocks.sort_values('avg_yr_returns', ascending = False).head(20)
top_20_moderate_growth = moderate_growth_stocks.sort_values('avg_yr_returns', ascending = False).head(20)
top_20_risky_growth = risky_growth_stocks.sort_values('avg_yr_returns', ascending = False).head(20)

### Time Horizon

In [619]:
def time_horizon():
    time_investing = input('How long would you like to invest for: ')
    while True:
        try:
            time_investing = int(time_investing)
            if type(time_investing) == int:
                break
        except ValueError:
            print('Please use digits')
            time_investing  = input('How long would you like to invest for: ')
    return time_investing

In [620]:
time_horizon = time_horizon()

How long would you like to invest for: 5


### User Input Choices

In [590]:
def user_input_choices():
    risk_tolerance = input(" ----------\n 1- Conservative\n 2- Moderate\n 3- Risky \n Please choose from the following: ").strip().lower()
    while True:
        if risk_tolerance == 'conservative' or risk_tolerance == '1':
            risk = conservative()
            break
        elif risk_tolerance == 'moderate' or risk_tolerance == '2': 
            risk = moderate()
            break
        elif risk_tolerance == 'risky' or risk_tolerance == '3':   
            risk = risky()
            break
        else:
            print("invalid input try again\n ------")
            risk_tolerance = input(" 1- Conservative\n 2- Moderate\n 3- Risky \n Please choose from the following: ").lower()
    risks = risk
    return risks

###  Risk Tolerance

In [621]:
def conservative():
    counter = 0
    counter_2 = 0
    value_stocks = []
    growth_stocks = []
    portfolio = []
    Y = []
    top_picks_value = top_20_conservative_value['symbol'].tolist()
    top_picks_growth = top_20_conservative_growth['symbol'].tolist()
    full_stock_value = [top_20_conservative_growth, top_20_conservative_value]
    full_stock_value = pd.concat(full_stock_value)
    print('------------------------')
    print(top_picks_value)
    while counter < 3:
        user_input = input('Please choose 3 from the top 20 Value Stocks: ').upper()
        if user_input in top_picks_value:
            counter += 1
            value_stocks.append(user_input)
            portfolio.append(user_input)
        else:
            print('Stock Not in List')
            print('------------------------')
            print(top_picks_value)
            continue
    print('------------------------')
    print(top_picks_growth)
    while counter_2 < 7:
        user_input_2 = input('Please choose 7 from the top 20 Growth Stocks: ').upper()
        if user_input_2 in top_picks_growth:
            counter_2 += 1
            growth_stocks.append(user_input_2)
            portfolio.append(user_input_2)
        else:
            print('Stock Not in List')
            print('------------------------')
            print(top_picks_growth)
            continue
    
    for tickers in portfolio:
        df = full_stock_value[full_stock_value['symbol'] == tickers]
        Y.append(df)
    
    new_port = pd.concat(Y)
    print(new_port)
        
    return new_port

In [622]:
def moderate():
    counter = 0
    counter_2 = 0
    value_stocks = []
    growth_stocks = []
    portfolio = []
    top_picks_value = top_20_moderate_value['symbol'].tolist()
    top_picks_growth = top_20_moderate_growth['symbol'].tolist()
    full_stock_value = [top_20_moderate_growth, top_20_moderate_value]
    full_stock_value = pd.concat(full_stock_value)
    print('------------------------')
    print(top_picks_value)
    while counter < 5:
        user_input = input(f'Please choose 5 from the top 20 Value Stocks: ').upper()
        if user_input in top_picks_value:
            counter += 1
            value_stocks.append(user_input)
            portfolio.append(user_input)
        else:
            print('Stock Not in List')
            print('------------------------')
            print(top_picks_value)
            continue
    print('------------------------')
    print(top_picks_growth)
    while counter_2 < 5:
        user_input_2 = input('Please choose 5 from the top 20 Growth Stocks: ').upper()
        if user_input_2 in top_picks_growth:
            counter_2 += 1
            growth_stocks.append(user_input_2)
            portfolio.append(user_input_2)
        else:
            print('Stock Not in List')
            print('------------------------')
            print(top_picks_growth)
            continue
    Y = []
    for tickers in portfolio:
        df = full_stock_value[full_stock_value['symbol'] == tickers]
        Y.append(df)
    
    new_port = pd.concat(Y)
    print(portfolio)

    return portfolio

In [623]:
def risky():
    counter = 0
    counter_2 = 0
    value_stocks = []
    growth_stocks = []
    portfolio = []
    top_picks_value = top_20_risky_value['symbol'].tolist()
    top_picks_growth = top_20_risky_growth['symbol'].tolist()
    full_stock_value = [top_20_risky_growth, top_20_risky_value]
    full_stock_value = pd.concat(full_stock_value)
    print('------------------------')
    print(top_picks_value)
    while counter < 3:
        user_input = input('Please choose 3 from the top 20 Value Stocks: ').upper()
        if user_input in top_picks_value:
            counter += 1
            value_stocks.append(user_input)
            portfolio.append(user_input)
        else:
            print('Stock Not in List')
            print('------------------------')
            print(top_picks_value)
            continue
    print('------------------------')
    print(top_picks_growth)
    while counter_2 < 8:
        user_input_2 = input('Please choose 8 from the top 20 Growth Stocks: ').upper()
        if user_input_2 in top_picks_growth:
            counter_2 += 1
            growth_stocks.append(user_input_2)
            portfolio.append(user_input_2)
        else:
            print('Stock Not in List')
            print('------------------------')
            print(top_picks_growth)
            continue
    Y = []
    for tickers in portfolio:
        df = full_stock_value[full_stock_value['symbol'] == tickers]
        Y.append(df)
    
    new_port = pd.concat(Y)
        
    return new_port   

### Recommender System

In [624]:
def processing_recommender_knn(portfolio, cluster_rec):
    model_knn = pickle.load(open("dash_package/pickles/tfidf.pkl", "rb"))
    cluster = cluster_rec
    cluster = cluster.sort_values('sharpe_ratio', ascending = True)
    stocking = int(cluster.iloc[0][0])
    rec_comp_main = cluser.drop(columns ='Unnamed: 0')
    rec_test = cluster.drop(columns = ['symbol','company','sector'])
    test = [stocking]
    dftest = rec_test.iloc[test,:]
    distances, indices = model_knn.kneighbors(rec_test, n_neighbors= 50)
    stock = pd.DataFrame(indices.T)[test]
    stock = stock.set_index(test)
    stock = stock.merge(clus, left_index = True, right_index = True)
    testing_2 = stock

In [625]:
def processing_recommender(portfolio, cluster_rec):
    cluster = portfolio
    cluster = cluster[cluster['sharpe_ratio'] >= 0]
    cluster = cluster.sort_values('sharpe_ratio', ascending = True)
    rec_test = cluster_rec.drop(columns = ['symbol','company','Price','sector'])
    rec_clust = portfolio.drop(columns = ['symbol','company','sector'])
    pip = rec_clust.sort_values('avg_yr_returns', ascending=False)
    pip = pip.index[0]
    inpt_idx = pip
    inpt = rec_test.iloc[inpt_idx]
    scores = pd.Series(index=rec_test.index)
    scores.iloc[inpt_idx] = -1
    for idx, stock in rec_test.drop(index=inpt_idx).iterrows():
        diff = 0
        for feature in stock.index:
            diff += (inpt.loc[feature]-stock.loc[feature])**2
        scores.loc[idx] = diff
    scores.sort_values()
    scores = pd.DataFrame(scores.sort_values())
    recommend_err = scores.merge(cluster_rec, left_index = True, right_index = True)
    recommend_err = recommend_err.rename({0: 'RMSE'}, axis = 1)
    testing = recommend_err.head(10)
    return testing

In [592]:
recommender = processing_recommender(portfolio, cluster_rec)

### Monte Carlo Simulation

In [626]:
def monte_carlo(portfolio, time_horizon):
    sim = pd.DataFrame()
    iterations = 5000

    for x in range(iterations):
        pv = float(portfolio['Price'].sum()) + 6
        expected_return = float(portfolio['avg_yr_returns'].mean())
        volatility = float(portfolio['yr_variance'].mean())
        time_horizon = time_horizon
        annual_investment = pv
        stream = []
        for i in range(time_horizon):
            end = round(pv * (1 + np.random.normal(expected_return,volatility)) + annual_investment,2)

            stream.append(end)

            pv = end
            
        sim[x] = stream

In [627]:
monte_carlo = monte_carlo(portfolio, time_horizon)

In [634]:
def simulator(monte_carlo, portfolio):
    
    
    price_per = float(portfolio['Price'].sum()) + 6
    expected_return = float(portfolio['avg_yr_returns'].mean())
    volatility = float(portfolio['yr_variance'].mean())
    print(f'price per year : {price_per}')
    print(f'expected return : {expected_return}')
    print(f'risk : {volatility}')
    print('------------------')
    amount = len(sim)-1
    ending_values = sim.loc[amount]
    p_tiles = np.percentile(ending_values,[5,10,15])
    for p in range(len(p_tiles)):
        l = [5,10,15]
        print("{}%-ile:  ".format(l[p]).rjust(15),"{}".format(locale.currency(p_tiles[p], grouping=True)))
    
    random_x = sim.index
    random_y0 = sim[1]
    random_y1 = sim[2]
    random_y2 = sim[3]
    random_y3 = sim[4]
    random_y4 = sim[5]

    # Create traces
    fig = go.Figure()
    fig.update_layout(title='Your Portfolio Outcome', 
                      xaxis_title = 'Years Invested',
                     yaxis_title = 'Return')
    fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                        mode='lines+markers',
                        name='stream_1'))
    fig.add_trace(go.Scatter(x=random_x, y=random_y1,
                        mode='lines+markers',
                        name='stream_2'))
    fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                        mode='lines+markers', 
                        name='stream_3'))

    fig.add_trace(go.Scatter(x=random_x, y=random_y3,
                        mode='lines+markers',
                        name='stream_4'))
    fig.add_trace(go.Scatter(x=random_x, y=random_y4,
                        mode='lines+markers', 
                        name='stream_5'))

    return fig 

In [635]:
simulate = simulator(monte_carlo, portfolio)

price per year : 1621.240116596222
expected return : 0.27741535105160764
risk : 0.3727017785146298
------------------
      5%-ile:   $1,487.85
     10%-ile:   $1,702.39
     15%-ile:   $1,874.44


###  Practice Testing

In [410]:
###  User Input - Time Horizon, Risk Tolerance

In [411]:
def time_horizon():
    time_investing = input('How long would you like to invest for: ')
    while True:
        try:
            time_investing = int(time_investing)
            if type(time_investing) == int:
                break
        except ValueError:
            print('Please use digits')
            time_investing  = input('How long would you like to invest for: ')
    return time_investing


def risk_tolerance():
    risk_tolerance = input(" ----------\n 1- Conservative\n 2- Moderate\n 3- Risky \n Please choose from the following: ").strip().lower()
    while True:
        if risk_tolerance == 'conservative' or risk_tolerance == '1':
            risk_tolerance = 30
            break
        elif risk_tolerance == 'moderate' or risk_tolerance == '2': 
            risk_tolerance = 50 
            break
        elif risk_tolerance == 'risky' or risk_tolerance == '3':   
            risk_tolerance = 80
            break
        else:
            print("invalid input try again\n ------")
            risk_tolerance = input(" 1- Conservative\n 2- Moderate\n 3- Risky \n Please choose from the following: ").lower()
    risks = float(risk_tolerance)/100
    return risks

In [ ]:
# risks = input('input your risk: ')
cluster = cluster_rec[cluster_rec['yr_variance'] <= risk_tolerance()]
cluster = cluster[cluster['sharpe_ratio'] >= 0]
cluster.sort_values('sharpe_ratio', ascending = True)
stocking = int(cluster.iloc[0][0])
#testing trial
rec_comp_main = cluster.drop(columns ='Unnamed: 0')
rec_test = rec_comp_main.drop(columns = ['symbol','company','sector'])
pip = rec_test.sort_values('avg_yr_returns', ascending=False)
pip = pip.index[0]
scaler = StandardScaler()
scaler.fit(rec_test)
model_knn = NearestNeighbors(metric = 'cosine', algorithm= 'brute')
model_knn.fit(rec_test)

# KNN recommender system
test = [4]
dftest = rec_test.iloc[test,:]
distances, indices = model_knn.kneighbors(rec_test, n_neighbors=30)
stock = pd.DataFrame(indices.T)[test]
stock = stock.set_index(test)
stock = stock.merge(rec_comp_main, left_index = True, right_index = True)
testing_2 = stock
testing_2

In [ ]:
cluster = cluster[cluster['yr_variance'] == risk_tolerance()]
cluster = cluster[cluster['sharpe_ratio'] >= 0]
cluster.sort_values('sharpe_ratio', ascending = True)
stocking = int(cluster.iloc[0][0])
#testing trial
rec_test = rec_comp_main.drop(columns = ['symbol', 'Price','company','sector'])
pip = rec_test.sort_values('avg_yr_returns', ascending=False)
pip = pip.index[0]

inpt_idx = pip
inpt = rec_test.loc[inpt_idx]
scores = pd.Series(index=rec_test.index)
scores.loc[inpt_idx] = -1
for idx, stock in rec_test.drop(index=inpt_idx).iterrows():
    diff = 0
    for feature in stock.index:
        diff += (inpt.loc[feature]-stock.loc[feature])**2
    scores.loc[idx] = diff
scores.sort_values()
scores = pd.DataFrame(scores.sort_values())
recommend_err = scores.merge(rec_comp_main, left_index = True, right_index = True)
recommend_err = recommend_err.rename({0: 'RMSE'}, axis = 1)
testing = recommend_err.head(4)

In [225]:
# Show user price paid per year
# Show user expected return
# Show user risk factor
price_per = float(testing['Price'].sum()) + 6
expected_return = float(testing['avg_yr_returns'].mean())
volatility = float(testing['yr_variance'].mean())
print(f'price per year : {price_per}')
print(f'expected return : {expected_return}')
print(f'risk : {volatility}')

price per year : 246.67980241775513
expected return : 0.25715072226272684
risk : 0.2784875816204984


In [235]:
pv = float(testing['Price'].sum()) + 6
expected_return = float(testing['avg_yr_returns'].mean())
volatility = float(testing['yr_variance'].mean())
time_horizon = time_horizon
annual_addition = pv

print("\tReturn", "\t\tEnding Value".rjust(18))
for year in range(time_horizon):
    market_return = np.random.normal(expected_return, volatility)
    fv = pv * (1 + market_return) + annual_addition
    print("\t{}".ljust(10).format(round(market_return,4)), "\t{}".rjust(10).format(locale.currency(fv, grouping=True)))
    pv = fv

	Return     		Ending Value
	0.4431               	$602.67
	0.207               	$974.09
	0.3047               	$1,517.54
	-0.0708               	$1,656.73
	0.431               	$2,617.47


In [245]:
# 4. Simulate portfolio ending market values 

sim = pd.DataFrame()
iterations = 5000


for x in range(iterations):
    pv = float(testing['Price'].sum()) + 6
    expected_return = float(testing['avg_yr_returns'].mean())
    volatility = float(testing['yr_variance'].mean())
    time_horizon = time_horizon
    annual_investment = pv
    stream = []
    for i in range(time_horizon):
        end = round(pv * (1 + np.random.normal(expected_return,volatility)) + annual_investment,2)
        
        stream.append(end)

        pv = end

    
    sim[x] = stream

In [246]:
# 5. Sample first five streams of annual ending values
first_five = list(range(5))
sim[first_five]

,0,1,2,3,4
0,463.09,516.32,528.41,576.76,650.52
1,863.94,875.74,822.22,931.89,1412.03
2,1384.23,1017.20,1641.84,1459.06,1806.27
3,2659.02,1545.62,3008.98,2135.89,1906.47
4,3307.45,2464.24,4498.86,2894.51,1830.76


In [247]:
random_x = sim.index
random_y0 = sim[1]
random_y1 = sim[2]
random_y2 = sim[3]
random_y3 = sim[4]
random_y4 = sim[5]

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='lines+markers',
                    name='stream_1'))
fig.add_trace(go.Scatter(x=random_x, y=random_y1,
                    mode='lines+markers',
                    name='stream_2'))
fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    mode='lines+markers', 
                    name='stream_3'))

fig.add_trace(go.Scatter(x=random_x, y=random_y3,
                    mode='lines+markers',
                    name='stream_4'))
fig.add_trace(go.Scatter(x=random_x, y=random_y4,
                    mode='lines+markers', 
                    name='stream_5'))

fig.show()

In [243]:
# 7. Generate summary statistics with numpy functions
amount = len(sim)-1
print("Count:", len(sim.loc[amount]))
print("Mean: ", locale.currency(np.mean(sim.loc[amount]),grouping=True))
print("SD: ",locale.currency(np.std(sim.loc[amount]),grouping=True))
print("Max: ",locale.currency(np.max(sim.loc[amount]), grouping=True))
print("Min: ", locale.currency(np.min(sim.loc[amount]), grouping=True))

Count: 5000
Mean:  $14,040.78
SD:  $4,429.33
Max:  $41,945.76
Min:  $4,291.82


In [249]:
#You can get a more comprehensive table of percentiles easily using numpy's percentile function
amount = len(sim)-1
ending_values = sim.loc[amount]
p_tiles = np.percentile(ending_values,[5,10,15,25,50,75])
for p in range(len(p_tiles)):
    l = [5,10,15,25,50,75]
    print("{}%-ile:  ".format(l[p]).rjust(15),"{}".format(locale.currency(p_tiles[p], grouping=True)))
#     print(f"{l[p]}%-ile:  ",f"{locale.currency(p_tiles[p], grouping=True)}")

      5%-ile:   $1,487.85
     10%-ile:   $1,702.39
     15%-ile:   $1,874.44
     25%-ile:   $2,145.32
     50%-ile:   $2,713.24
     75%-ile:   $3,392.91
